let's start off by getting the libraries for our webscraping and for word2vec

In [22]:
from selenium.webdriver import Chrome
import time
import random
import json
import pandas as pd
import string
import re
import gensim
import numpy as np
import pickle
import functions as fc

set up for our web scraper and ease of access to a web blocker to install on the selenium instance

In [2]:
# browser = Chrome()


# urls = ["https://www.allrecipes.com/recipes/76/appetizers-and-snacks/",
#         "https://www.allrecipes.com/recipes/156/bread/",
#         "https://www.allrecipes.com/recipes/78/breakfast-and-brunch/",
#         "https://www.allrecipes.com/recipes/276/desserts/cakes/",
#         "https://www.allrecipes.com/recipes/201/meat-and-poultry/chicken/",
#         "https://www.allrecipes.com/recipes/362/desserts/cookies/",
#         "https://www.allrecipes.com/recipes/430/seafood/shellfish/shrimp/",
#         "https://www.allrecipes.com/recipes/253/everyday-cooking/slow-cooker/",
#         "https://www.allrecipes.com/recipes/1227/everyday-cooking/vegan/",
#         "https://www.allrecipes.com/recipes/86/world-cuisine/"]

# browser.set_page_load_timeout(300)
# browser.get('https://chrome.google.com/webstore/detail/adblock/gighmmpiobklfepjocnamgkkbiglidom')

let the scraping commence!

In [3]:
# for url in urls:
#     print(url)
#     browser.get(url)

#     for _ in range(6):
#         browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(3 + random.random()*10)

#     link_list = browser.find_elements_by_xpath('//*[contains(@class,"fixed-recipe-card__info")]/a')

#     recipe_page_list = [link.get_attribute('href') for link in link_list]

#     len(recipe_page_list)

#     with open('recipes_list.json', 'a') as f:


#         for recipe in recipe_page_list:

#             browser.get(recipe)

#             recipe_dict = {}
            
#             try:
#                 recipe_dict['title'] = browser.find_element_by_xpath('//*[@id="recipe-main-content"]').text
#             except:
#                 recipe_dict['title'] = browser.find_element_by_xpath('//*[contains(@class, "intro article-info")]/h1').text
            
#             try:
#                 lala_list = browser.find_elements_by_xpath('//*[contains(@id,"lst_ingred")]')
#                 item_list = []
#                 for element in lala_list:

#                     item_list += element.find_elements_by_css_selector('span')

#                 ingredients = [item.text for item in item_list[:-1]]
#                 recipe_dict['ingredients'] = ingredients
            
#             except:
#                 lala_list = browser.find_elements_by_xpath('//*[contains(@class, "ingredients-item-name")]')

#                 ingredients = [item.text for item in lala_list]
#                 recipe_dict['ingredients'] = ingredients

#             json.dump(recipe_dict, f)
#             f.write('\n')
#             time.sleep(3 + random.random()*10)

In [23]:
#bring in the json data line by line
data = []
with open('recipes_list.json') as file:
    for line in file:
        data.append(json.loads(line))


In [24]:
df = pd.DataFrame(data)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 2 columns):
ingredients    929 non-null object
title          929 non-null object
dtypes: object(2)
memory usage: 14.6+ KB


In [26]:

indices_list = []
for i in range(len(df.ingredients)):
    
    
        try: 
            if df.ingredients[i][0] == '':
                print(df.ingredients[i])
                indices_list.append(i)
                
        except IndexError:
            print(df.ingredients[i])
            indices_list.append(i)


print(indices_list)

[]
['', '', '', '', '', '', '', '', '', '']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[60, 136, 217, 220, 222, 251, 436, 441, 509, 710, 713, 731, 778, 788, 867]


In [27]:
to_be_reindexed_df = df.drop(indices_list)

clean_df = to_be_reindexed_df.reset_index(drop=True)
clean_df.head()

,ingredients,title
0,"[2 pounds unpeeled large shrimp, 2 tablespoons...",Grilled Shrimp Scampi Cocktail
1,"[cooking spray, 24 (1 inch) cubes fresh pineap...",Easy Rumaki with Pineapple
2,"[2 tablespoons butter, 2 tablespoons minced gr...",Perfect Crab-Stuffed Mushrooms
3,"[3 cups diced cooked rotisserie chicken, 2 (8 ...",Baked Buffalo Chicken Dip
4,"[24 slices French baguette, 1 tablespoon butte...",Strawberry Bruschetta


In [28]:
clean_df.ingredients[0]

['2 pounds unpeeled large shrimp',
 '2 tablespoons olive oil',
 '1 teaspoon minced garlic',
 '1/4 teaspoon salt',
 '1/8 teaspoon freshly ground black pepper',
 '1 cup cocktail sauce',
 '1 lemon, cut into wedges']

In [29]:
clean_df.ingredients = clean_df.ingredients.map(fc.clean_ingredients)

In [30]:
model = gensim.models.Word2Vec(clean_df.ingredients, sg=1)

model.train(clean_df.ingredients,
            total_examples=model.corpus_count,
            epochs=model.epochs)

model.corpus_total_words

model.wv

In [48]:
model.wv.most_similar(positive="bacon".lower().split() + "honey sauce".lower().split() + "cheese".lower().split() )

[('crabmeat', 0.9901946187019348),
 ('sage', 0.9899771213531494),
 ('rounds', 0.9871524572372437),
 ('chops', 0.9853564500808716),
 ('sherry', 0.9851032495498657),
 ('broccoli', 0.9848699569702148),
 ('turkey', 0.9838480949401855),
 ('cucumber', 0.9812321662902832),
 ('chestnuts', 0.9806040525436401),
 ('Sauce', 0.9805401563644409)]

In [21]:
#pickle.dump(model, open('ingred_model.pkl', 'wb'))

#pickle.dump(clean_df.ingredients, open('ingredients.pkl', 'wb'))

#pickle.dump(clean_df.title, open('title.pkl', 'wb'))

In [15]:
ingred_model = pickle.load(open('ingred_model.pkl', 'rb'))

In [32]:
fc.sound_tasty(model, clean_df.ingredients, clean_df.title)

Choose an ingredient: milk
Choose another: eggs
And another: bacon
Last one: cheese
How adventurous are you feeling?
            low, medium, or high? high


I'll get food pretty different from what you're used too



 Ingredients like yours: 

 ['paprika', 'clove', 'leaf', 'green', 'oregano', 'tomato', 'corn', 'peppers', 'chile', 'ginger', 'chili', 'cilantro', 'jalapeno', 'garlic', 'tomatoes', 'bay', 'kidney', 'beans', 'bell', 'cumin']


 How about?  

 Papa Funk's Campfire Chili 

 ['beef', 'sausage', 'tomato', 'sauce', 'red', 'kidney', 'beans', 'plum', 'tomatoes', 'celery', 'green', 'bell', 'peppers', 'red', 'kidney', 'beans', 'white', 'kidney', 'beans', 'beer', 'Yuengling', 'onion', 'Vidalia', 'red', 'bell', 'pepper', 'tomato', 'paste', 'green', 'chile', 'peppers', 'chili', 'powder', 'chipotle', 'chile', 'peppers', 'adobo', 'sauce', 'Goya', 'adobo', 'sauce', 'chipotle', 'peppers', 'Goya', 'cumin', 'barbeque', 'sauce'] 




Sound tasty? yes/no: no


 Fine, I'll find you something el